In [1]:
import os
import openai
import json
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("facebook/flores", "all")  # Loads all languages

In [2]:
language_mapping = pd.read_csv('language_mapping.csv')
language_mapping_index = language_mapping.set_index('FLORES-200 code')['Language'].to_dict()
all_languages = list(language_mapping_index.keys())

In [3]:
flores_l = {'afr_Latn': 'Afrikaans',
 'amh_Ethi': 'Amharic',
 'arb_Arab': 'Arabic',
 'asm_Beng': 'Assamese',
 'ast_Latn': 'Asturian',
 'azj_Latn': 'Azerbaijani',
 'bel_Cyrl': 'Belarusian',
 'ben_Beng': 'Bengali',
 'bos_Latn': 'Bosnian',
 'bul_Cyrl': 'Bulgarian',
 'cat_Latn': 'Catalan',
 'ceb_Latn': 'Cebuano',
 'ces_Latn': 'Czech',
 'ckb_Arab': 'Kurdish',
 'cym_Latn': 'Welsh',
 'dan_Latn': 'Danish',
 'deu_Latn': 'German',
 'ell_Grek': 'Greek',
 'eng_Latn': 'English',
 'est_Latn': 'Estonian',
 'fin_Latn': 'Finnish',
 'fra_Latn': 'French',
 'gaz_Latn': 'Oromo',
 'gle_Latn': 'Irish',
 'glg_Latn': 'Galician',
 'guj_Gujr': 'Gujarati',
 'hau_Latn': 'Hausa',
 'heb_Hebr': 'Hebrew',
 'hin_Deva': 'Hindi',
 'hrv_Latn': 'Croatian',
 'hun_Latn': 'Hungarian',
 'hye_Armn': 'Armenian',
 'ibo_Latn': 'Igbo',
 'ind_Latn': 'Indonesian',
 'isl_Latn': 'Icelandic',
 'ita_Latn': 'Italian',
 'jav_Latn': 'Javanese',
 'jpn_Jpan': 'Japanese',
 'kam_Latn': 'Kamba',
 'kan_Knda': 'Kannada',
 'kat_Geor': 'Georgian',
 'kaz_Cyrl': 'Kazakh',
 'kea_Latn': 'Kabuverdianu',
 'khk_Cyrl': 'Mongolian',
 'khm_Khmr': 'Khmer',
 'kor_Hang': 'Korean',
 'lao_Laoo': 'Lao',
 'lin_Latn': 'Lingala',
 'lit_Latn': 'Lithuanian',
 'ltz_Latn': 'Luxembourgish',
 'lug_Latn': 'Ganda',
 'luo_Latn': 'Luo',
 'lvs_Latn': 'Latvian',
 'mal_Mlym': 'Malayalam',
 'mar_Deva': 'Marathi',
 'mkd_Cyrl': 'Macedonian',
 'mlt_Latn': 'Maltese',
 'mri_Latn': 'Maori',
 'mya_Mymr': 'Burmese',
 'nld_Latn': 'Dutch',
 'nob_Latn': 'Norwegian',
 'npi_Deva': 'Nepali',
 'nso_Latn': 'Sotho',
 'nya_Latn': 'Nyanja',
 'oci_Latn': 'Occitan',
 'ory_Orya': 'Odia',
 'pan_Guru': 'Punjabi',
 'pbt_Arab': 'Pashto',
 'pes_Arab': 'Persian',
 'pol_Latn': 'Polish',
 'por_Latn': 'Portuguese',
 'ron_Latn': 'Romanian',
 'rus_Cyrl': 'Russian',
 'slk_Latn': 'Slovak',
 'slv_Latn': 'Slovenian',
 'smo_Latn': 'Samoan',
 'sna_Latn': 'Shona',
 'snd_Arab': 'Sindhi',
 'som_Latn': 'Somali',
 'spa_Latn': 'Spanish',
 'srp_Cyrl': 'Serbian',
 'swe_Latn': 'Swedish',
 'swh_Latn': 'Swahili',
 'tam_Taml': 'Tamil',
 'tel_Telu': 'Telugu',
 'tgk_Cyrl': 'Tajik',
 'tgl_Latn': 'Tagalog',
 'tha_Thai': 'Thai',
 'tur_Latn': 'Turkish',
 'ukr_Cyrl': 'Ukrainian',
 'umb_Latn': 'Umbundu',
 'urd_Arab': 'Urdu',
 'uzn_Latn': 'Uzbek',
 'vie_Latn': 'Vietnamese',
 'wol_Latn': 'Wolof',
 'xho_Latn': 'Xhosa',
 'yor_Latn': 'Yoruba',
 'zho_Hans': 'Chinese',
 'zho_Hant': 'Chinese',
 'zsm_Latn': 'Malay',
 'zul_Latn': 'Zulu'}

In [4]:
prompt_lid= "You are an assistant specialized in identifying written languages. Please identify the language in the text below. Return only the name of the language, no other text.\n Text: {}"


In [5]:
languages = list(flores_l.keys())

In [6]:
import time

import requests
url = "http://localhost:8000/v1/chat/completions"
headers = {"Content-Type": "application/json"}

def get_gemma3_reply(prompt):
    data = {
        "model": "google/gemma-3-27b-it",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    completion = response.json()

    content = completion["choices"][0]["message"]["content"]
    return content

def get_category(prompt, max_retries=3, backoff_factor=2):
    for attempt in range(max_retries):
        try:
            response = get_gemma3_reply(prompt)
            if response is not None:
                return response
            else:
                return "invalid"
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < max_retries - 1:
                sleep_time = backoff_factor ** attempt
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                print("Max retries reached. Returning failure label.")
                return "invalid"  
                

In [7]:
languages_to_run = languages[0:34]

In [8]:
from tqdm import tqdm
result_categories = {}
result_accuracies = {}
gpt_replies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0
    
    result_categories[language_code]=[]
    gpt_replies[language_code] = []

    language_name = flores_l[language_code]
    
    size = 1012
    for i in tqdm(range(size)): #length of devtest
        sentence = dataset['devtest'][i]["sentence_{}".format(language_code)]
        prompt = prompt_lid.format(sentence)
        # print(prompt)
        reply = get_category(prompt)
        gpt_replies[language_code].append(reply)

        category = ""
        if language_name.lower() in reply.lower():
            category = language_name
        result_categories[language_code].append(category)
        if category == language_name:
            accurate+=1
            
    result_accuracies[language_code] = accurate

    df = pd.DataFrame({
    "gemma3_answer": result_categories[language_code]
    })

    # Save to CSV
    df.to_csv("lid_predicted_gemma3/{}.csv".format(language_code), index=False)

    df2 = pd.DataFrame({
    "gemma3_reply": gpt_replies[language_code]
    })

    # Save to CSV
    df2.to_csv("lid_replies_gemma3/{}.csv".format(language_code), index=False)
    print(accurate)
print(result_accuracies)

afr_Latn


 10%|████████████████▊                                                                                                                                                       | 1/10 [00:00<00:05,  1.76it/s]

Afrikaans



 20%|█████████████████████████████████▌                                                                                                                                      | 2/10 [00:01<00:04,  1.89it/s]

Afrikaans



...

Arabic



 40%|███████████████████████████████████████████████████████████████████▏                                                                                                    | 4/10 [00:01<00:02,  2.61it/s]

Arabic



 40%|███████████████████████████████████████████████████████████████████▏                                                                                                    | 4/10 [00:01<00:02,  2.04it/s]


KeyboardInterrupt: 